## Name : Sahil Gupta
## Roll No. : D21030
### Part 1 : <b> [Content-Based Filtering](#5)</b>
### Part 2 : <b> [Collaborative Filtering](#6)</b>

### <a id = '5'>Part 1 : Movie Recommender System - (Content-Based Filtering)</a>

####  <b>[1. In this, I have also used the year of release data which is given along with the title of the movie to extract the release year of the movie and calculate the years elapsed since release date.](#1)</b>


#### <b>[2. I have used this year elapsed information in the movie-movie cosine similarity.](#2)</b>

### 0. Libraries :

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

### 1. Dataset 
#### 1.1 Movies

In [2]:
#Reading the movies data
movies = pd.read_csv(r"C:\Users\Lenovo\Documents\Codes\AML\Datasets\movies.csv")

In [3]:
movies.sample(2)

,movieId,title,genres
6385,50440,Primeval (2007),Horror|Thriller
6983,66798,"Pink Panther 2, The (2009)",Adventure|Comedy|Mystery


#### 1.2 Ratings

In [4]:
#Reading the ratings data
ratings = pd.read_csv(r"C:\Users\Lenovo\Documents\Codes\AML\Datasets\ratings.csv")

In [5]:
ratings.sample(2)

,userId,movieId,rating,timestamp
2377,19,494,2.0,965711156
25355,177,5957,4.0,1435525349


In [6]:
#movies watched by user 2 with ratings > 5
ratings.loc[(ratings.userId==2)&(ratings.rating>=5),:]  

,userId,movieId,rating,timestamp
241,2,60756,5.0,1445714980
248,2,80906,5.0,1445715172
250,2,89774,5.0,1445715189
254,2,106782,5.0,1445714966
259,2,122882,5.0,1445715272
260,2,131724,5.0,1445714851


In [7]:
ratings.loc[(ratings.userId==2)&(ratings.rating>=5),:].shape

(6, 4)

### 2. Data Preparation 

#### 2.1 Creating the Movie - Genre binary Matrix

The below code is used to create Movie - Genre binary Matrix to seperate all the generes to which a movie belongs.  
It creates a column for each genre present in the dataset and marks 1 against that movie if the particular column genre is present in the datset. It is like one hot encoding of geners by splitting them.

In [8]:
#Identifying list of unique genres
genres_list = []
for genre in movies['genres'] :
    genre_splitted = genre.split('|')
    for item in genre_splitted :
        if item not in genres_list :
            genres_list.append(item)
            
#Creating individual column for each genre

#Initial values is set to zero for all new columns
for genre in genres_list :
    movies[genre] = 0
    
#Using indicator to indicate presence of genre 

# Movie[movie, Genre] = 1, if Genre is present in movie

for index , row in movies.iterrows() :
    for current_genre in genres_list :
        if current_genre in row['genres'] :
            movies.loc[index, current_genre] = 1

#### <a id = '1'>2.2 Extracting the year of release information</a>


In [9]:
#Extracting the year of release information
int(movies.title[0][-5:-1])

1995

**In this dataset we can see that the movie title is followed by the year of release at the end. This information can be used in our recommendation system.Therefore we extract the year information and make a new column in our dataset named 'year_of_release' to store this information.**

In [10]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies['year_of_release'] = movies.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies['year_of_release'] = movies.year_of_release.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
#movies['title'] = movies.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies['title'] = movies['title'].apply(lambda x: x.strip())

In [11]:
movies.head()   #printing the head after extracting the year

,movieId,title,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed),year_of_release
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,1995
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1995


#### Here we check for the null values in the year_of_release of column.

In [12]:
movies[movies.year_of_release.isnull()]

,movieId,title,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed),year_of_release
6059,40697,Babylon 5,Sci-Fi,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,NaN
9031,140956,Ready Player One,Action|Sci-Fi|Thriller,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,NaN
9091,143410,Hyena Road,(no genres listed),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
9138,147250,The Adventures of Sherlock Holmes and Doctor W...,(no genres listed),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
9179,149334,Nocturnal Animals,Drama|Thriller,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,NaN
9259,156605,Paterson,(no genres listed),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
9367,162414,Moonlight,Drama,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,NaN
9448,167570,The OA,(no genres listed),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
9514,171495,Cosmos,(no genres listed),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
9515,171631,Maria Bamford: Old Baby,(no genres listed),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN


In [13]:
len(movies[movies.year_of_release.isnull()])   #number of movies with nan as the year of release

13

From above we observe that there are 13 movies present with year_of_release as NaN.  
These need to be imputed.  
We use the median of the year_of_release to impute these years.  
Further we observe that the 'movieId' = 171749 has the year_of_release as (2006–2007) which was not subseted in our regex evaluation.  
We manually put 2007 as the year_of_release for that movie.

In [14]:
movies.year_of_release[movies.movieId == 171749] = 2007  #replacing the year_of_release to 2007

In [15]:
movies.year_of_release[movies.movieId == 171749]   #checking

9518    2007
Name: year_of_release, dtype: object

In [16]:
movies[movies.year_of_release.isnull()]    #rechecking the null values

,movieId,title,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed),year_of_release
6059,40697,Babylon 5,Sci-Fi,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,NaN
9031,140956,Ready Player One,Action|Sci-Fi|Thriller,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,NaN
9091,143410,Hyena Road,(no genres listed),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
9138,147250,The Adventures of Sherlock Holmes and Doctor W...,(no genres listed),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
9179,149334,Nocturnal Animals,Drama|Thriller,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,NaN
9259,156605,Paterson,(no genres listed),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
9367,162414,Moonlight,Drama,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,NaN
9448,167570,The OA,(no genres listed),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
9514,171495,Cosmos,(no genres listed),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
9515,171631,Maria Bamford: Old Baby,(no genres listed),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN


In [17]:
#finding the median of the year_of_release using the year
med = []                                               #empty list to store the values of year_of_release
for i in range(len(movies)):                           #iterating the movies dataset
    if type(movies.year_of_release[i]) is str :        #condition to check that values are not NaN and extract them
        med.append(int(movies.year_of_release[i]))     #appending the year_of_release values to the list in integer form

np.median(med)                                         #calculating the median

1999.0

We impute the remaining NaN values with median of year_of_release i.e. 1999

In [18]:
movies.year_of_release[movies.year_of_release.isnull()] = int(np.median(med))   #imputing the null values

In [19]:
movies[movies.year_of_release.isnull()].sum()    #rechecking the null values

movieId               0.0
title                 0.0
genres                0.0
Adventure             0.0
Animation             0.0
Children              0.0
Comedy                0.0
Fantasy               0.0
Romance               0.0
Drama                 0.0
Action                0.0
Crime                 0.0
Thriller              0.0
Horror                0.0
Mystery               0.0
Sci-Fi                0.0
War                   0.0
Musical               0.0
Documentary           0.0
IMAX                  0.0
Western               0.0
Film-Noir             0.0
(no genres listed)    0.0
year_of_release       0.0
dtype: float64

In [20]:
movies.year_of_release[1]    #type checking the year_of_release column

'1995'

#### As we see that the year_of_release is a str column we convert into int.

In [21]:
# Converting to int
#for i in range(len(movies)):
#    movies.year_of_release[i] = int(movies.year_of_release[i])

In [22]:
type(movies.year_of_release[1])  #checking the type

str

In [23]:
# Converting to int
movies["year_of_release"] = movies["year_of_release"].apply(pd.to_numeric)

In [24]:
movies.dtypes

movieId                int64
title                 object
genres                object
Adventure              int64
Animation              int64
Children               int64
Comedy                 int64
Fantasy                int64
Romance                int64
Drama                  int64
Action                 int64
Crime                  int64
Thriller               int64
Horror                 int64
Mystery                int64
Sci-Fi                 int64
War                    int64
Musical                int64
Documentary            int64
IMAX                   int64
Western                int64
Film-Noir              int64
(no genres listed)     int64
year_of_release        int64
dtype: object

In [25]:
movies.head()

,movieId,title,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed),year_of_release
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,1995
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1995


In [26]:
movies.describe()   #description of the transformed data set

,movieId,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,...,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed),year_of_release
count,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,...,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000
mean,42200.353623,0.129645,0.062718,0.068158,0.385547,0.079963,0.163827,0.447649,0.187641,0.123075,...,0.058817,0.100595,0.039212,0.034285,0.045165,0.016218,0.017142,0.008930,0.003490,1994.620304
std,52160.494854,0.335930,0.242468,0.252031,0.486749,0.271250,0.370137,0.497277,0.390445,0.328541,...,0.235295,0.300808,0.194108,0.181968,0.207677,0.126321,0.129808,0.094083,0.058976,18.523910
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1902.000000
25%,3248.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1988.000000
50%,7300.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1999.000000
75%,76232.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2008.000000
max,193609.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2018.000000


In [27]:
#droping the columns MovieID, title, genres and year of release and select all other binary columns
movies_sub = movies.iloc[:,3:-1]
movies_sub.sample(2)

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
6933,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
9304,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### <a id = '2'> 2.3 Creating column to get number of years elapsed since the movie was released.</a>

In [28]:
#Creating column to get number of years elapsed since the movie was released.
movies_sub["years"] = (2022 - movies["year_of_release"])
movies_sub.head(2)

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,...,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed),years
0,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,27
1,1,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,27


##### As the years column is having large values compared to other columns, I am scaling it down by a factor of 10.

In [29]:
movies_sub["years"] = movies_sub["years"]/10
movies_sub.head(2)

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,...,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed),years
0,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.7
1,1,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.7


#### 2.4 Creating a Movie-Movie similarity matrix(using cosine similarity)

In [30]:
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
#Creating a Movie Movie similarity matrix 
cos_movies = pd.DataFrame (cosine_similarity (movies_sub))
cos_movies.index = movies ['title']
cos_movies.columns = movies ['title']

In [32]:
cos_movies.head(2)

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Gintama: The Movie (2010),anohana: The Flower We Saw That Day - The Movie (2013),Silver Spoon (2014),Love Live! The School Idol Movie (2015),Jon Stewart Has Left the Building (2015),Black Butler: Book of the Atlantic (2017),No Game No Life: Zero (2017),Flint (2017),Bungo Stray Dogs: Dead Apple (2018),Andrew Dice Clay: Dice Rules (1991)
title,,,,,,,,,,,,,,,,,,,,,
Toy Story (1995),1.000000,0.915022,0.775837,0.737176,0.821299,0.648252,0.775837,0.869425,0.722228,0.737176,...,0.640849,0.583667,0.554765,0.675350,0.441665,0.601892,0.688290,0.344431,0.403702,0.820552
Jumanji (1995),0.915022,1.000000,0.745610,0.708455,0.789301,0.708455,0.745610,0.950167,0.789301,0.805637,...,0.433050,0.451903,0.414423,0.482682,0.482682,0.355358,0.406367,0.376419,0.229081,0.801051


In [33]:
cos_movies.shape

(9742, 9742)

In [34]:
cos_movies.describe()

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Gintama: The Movie (2010),anohana: The Flower We Saw That Day - The Movie (2013),Silver Spoon (2014),Love Live! The School Idol Movie (2015),Jon Stewart Has Left the Building (2015),Black Butler: Book of the Atlantic (2017),No Game No Life: Zero (2017),Flint (2017),Bungo Stray Dogs: Dead Apple (2018),Andrew Dice Clay: Dice Rules (1991)
count,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,...,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000
mean,0.684431,0.694389,0.766914,0.779932,0.792288,0.722215,0.766914,0.721640,0.764485,0.722607,...,0.524769,0.536007,0.581519,0.472036,0.471445,0.321795,0.329940,0.500224,0.277495,0.797322
std,0.133107,0.145009,0.162341,0.151491,0.167365,0.141129,0.162341,0.153132,0.158130,0.134833,...,0.114856,0.136817,0.157778,0.114897,0.134989,0.127677,0.135452,0.186967,0.114048,0.168126
min,0.151043,0.165070,0.155988,0.165070,0.165128,0.165070,0.155988,0.173728,0.183908,0.189397,...,0.100901,0.094542,0.096561,0.100981,0.100981,0.047565,0.048839,0.078750,0.053376,0.167587
25%,0.621230,0.612425,0.689612,0.712654,0.707630,0.649331,0.689612,0.633617,0.675720,0.649331,...,0.460179,0.459151,0.475312,0.405499,0.394972,0.220796,0.236261,0.370941,0.214013,0.704207
50%,0.718837,0.739396,0.815380,0.824005,0.842867,0.762307,0.815380,0.771026,0.816206,0.759936,...,0.530941,0.526327,0.600012,0.499135,0.494066,0.329608,0.271371,0.433861,0.249756,0.851234
75%,0.775837,0.803383,0.878464,0.891799,0.914430,0.819248,0.878464,0.840383,0.888720,0.816929,...,0.596093,0.636916,0.657129,0.543002,0.540678,0.392763,0.422598,0.664703,0.269267,0.923703
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


#### 2.5. Creating a movielD and MovieName dataset

In [35]:
movie_list = movies [["movieId", "title"]] 
movie_list.head() 

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


### 3. Finding user preferences

This function is used to list the top rated movies by a user.  
As we are filtering on the content liked by the user, we will only suggest top rated movies from the similar user.

In [36]:
def top_movies (user_id, top_n=5):
    max_rating = max(ratings.loc [(ratings.userId==user_id), "rating"]) 
    if max_rating < 4:
        print("None liked so far!")
        return (None)
    else :
        movie_5 = ratings.loc[(ratings.userId==user_id) & (ratings.rating==5), "movieId"] 
        movie_4 = ratings.loc[(ratings.userId==user_id) & (ratings.rating==4), "movieId"]
        
        #If number of 5-rated movies are more than top_n, then return a list of randomly selected movie
        if len(movie_5) > top_n:
            return (list (np.random.choice (movie_5,top_n, replace = False)))  #replace = False to get unique movies
        
        #If the total number of 5-rated movies & 4-rated movies are less than top_n, then return a list of movie rated 4  and 5
        elif len(movie_5) + len(movie_4) < top_n:
            return (list(movie_5)+list(movie4))
        
        #otherwise return a list containing all the 5-rated movies and a few random selection 
        else: 
            return(list (movie_5) + list (np.random.choice (movie_4, top_n-len(movie_5), replace = False))) #replace = False to get unique movies

In [37]:
top_movies(10,6)

[8869, 81845, 91529, 136020, 96079, 140110]

### 4. Finding Similar movies

This function is used to find the similar movies to the entered movie using thecosine similarity matrix.  
We have set a default minimum similarity of 0.95. This similarity value can be increased or decreased as per the user depending on the number of similar movies required by the user to recommend.

In [38]:
def get_similar_movies (movie_id, sim=0.95):

    #Get the movie title for the given movie id

    title= movie_list.loc [movie_list.movieId==movie_id, "title"]
    title = np.array(title)[0]

    #Extract the row containing the movie ID from the similarity matrix 
    similarity= cos_movies.loc [title,:]

    #scores of all movies
    scores = np.array(similarity)

    #Related movies
    related= cos_movies.loc[title, :][cos_movies.loc [title,:] >= sim].index

    return(list(related))

In [39]:
#to find the title of the movie_id from the movie_list created by subsetting the movies data
title = movie_list.loc[movie_list.movieId==2291, "title"]
title = np.array(title)[0]
title

'Edward Scissorhands (1990)'

In [40]:
len(get_similar_movies(2291,0.99))  #this will return the number of similar movies of a user.
#We can vary the similarity change the number of similar movies

8

In [41]:
len(get_similar_movies(49,1))  #this will return the number of similar movies of a user.
#We can vary the similarity change the number of similar movies

19

### 5. Movie Recommendation

#### 5.1. Get the user watchlist
You do not want to recommend the same movies to the users which he/she has already watched


In [42]:
#The below function will return the ID of all the movies watched by an user 
#def user_watch_list(user_id): 
#    index=list(ratings.loc[ratings.userId==user_id, "movieId"].index) 
#    return(list(movie_list["title"][index]))


In [43]:
#The below function will return the ID of all the movies watched by an user 
def user_watch_list(user_id): 

    id=list(ratings.loc[ratings.userId==user_id, "movieId"])   #creating a list of movie id watched by particular usr
    watched_user = list()                                    #empty list to store movie title
    for j in id :                              #loop to extract all the movie title watched by the user from movies data
        watched_user.append((movie_list.loc[movie_list.movieId == j, ["title"]].iloc[0,0])) 
    
    return(watched_user)

In [44]:
user_watch_list(278)       #sample run to check

['Usual Suspects, The (1995)',
 'To Wong Foo, Thanks for Everything! Julie Newmar (1995)',
 'Shawshank Redemption, The (1994)',
 "Schindler's List (1993)",
 'Island of Dr. Moreau, The (1996)',
 'Nightmare on Elm Street, A (1984)',
 'Ice Storm, The (1997)',
 'Jerk, The (1979)',
 'Sixteen Candles (1984)',
 'Big Chill, The (1983)',
 'Desperately Seeking Susan (1985)',
 'Fletch (1985)',
 'Thin Red Line, The (1998)',
 'Awakenings (1990)',
 'Cider House Rules, The (1999)',
 'Fast Times at Ridgemont High (1982)',
 'Muppet Movie, The (1979)',
 'Planes, Trains & Automobiles (1987)',
 'City Slickers (1991)',
 'Simpsons Movie, The (2007)']

#### 5.2. Content-Based Recommender System

In [45]:
#For a given user get the list of top-N most preferred_movies
N=5
User = 2
top = top_movies(User, N)
print(top)                     #this returns the movie id

[60756, 122882, 80906, 131724, 89774]


In [46]:
movie_list.loc[movie_list.movieId==top[1],:]    #name of the top[1] movie watched by the user

,movieId,title
8681,122882,Mad Max: Fury Road (2015)


In [47]:
#Get the most similar movie for the ith movie in the above List (LIST A) 
i = 4              #set any value of i to get 1 of the movies from top_N preffered movies of the user
sim_movies = get_similar_movies(top[i],0.99)   #we need to vary the similarity to change the number of similar movies
sim_movies

['Recount (2008)',
 'Frozen River (2008)',
 'Somers Town (2008)',
 'John Adams (2008)',
 'W. (2008)',
 'Secret Life of Bees, The (2008)',
 'Wave, The (Welle, Die) (2008)',
 'Express, The (2008)',
 'Tokyo! (2008)',
 'Class, The (Entre les murs) (2008)',
 'Milk (2008)',
 'Hunger (2008)',
 'Frost/Nixon (2008)',
 'Seven Pounds (2008)',
 'Wrestler, The (2008)',
 'Departures (Okuribito) (2008)',
 'Divo, Il (2008)',
 'Damned United, The (2009)',
 'Crossing Over (2009)',
 'Powder Blue (2009)',
 "My Sister's Keeper (2009)",
 'Still Walking (Aruitemo aruitemo) (2008)',
 'Coco Before Chanel (Coco avant Chanel) (2009)',
 'Misfortunates, The (De helaasheid der dingen) (2009)',
 'Private Lives of Pippa Lee, The (2009)',
 'Precious (2009)',
 'Blind Side, The  (2009)',
 'Shrink (2009)',
 'Single Man, A (2009)',
 'Invictus (2009)',
 "Hachiko: A Dog's Story (a.k.a. Hachi: A Dog's Tale) (2009)",
 'Temple Grandin (2010)',
 'Little Ashes (2008)',
 'Disgrace (2008)',
 'Enter the Void (2009)',
 'Endgame (200

In [48]:
len(sim_movies)    #the number of similar movies

124

In [49]:
#Get the watchlist of the user (LIST B) 
user_watch_list (User)

['Shawshank Redemption, The (1994)',
 'Tommy Boy (1995)',
 'Good Will Hunting (1997)',
 'Gladiator (2000)',
 'Kill Bill: Vol. 1 (2003)',
 'Collateral (2004)',
 'Talladega Nights: The Ballad of Ricky Bobby (2006)',
 'Departed, The (2006)',
 'Dark Knight, The (2008)',
 'Step Brothers (2008)',
 'Inglourious Basterds (2009)',
 'Zombieland (2009)',
 'Shutter Island (2010)',
 'Exit Through the Gift Shop (2010)',
 'Inception (2010)',
 'Town, The (2010)',
 'Inside Job (2010)',
 'Louis C.K.: Hilarious (2010)',
 'Warrior (2011)',
 'Dark Knight Rises, The (2012)',
 'Girl with the Dragon Tattoo, The (2011)',
 'Django Unchained (2012)',
 'Wolf of Wall Street, The (2013)',
 'Interstellar (2014)',
 'Whiplash (2014)',
 'The Drop (2014)',
 'Ex Machina (2015)',
 'Mad Max: Fury Road (2015)',
 'The Jinx: The Life and Deaths of Robert Durst (2015)']

In [50]:
#Get the list of movies in List A which are not in List B 
#That is the movies in the similarity list which the user has not watched yet

not_watched = set(sim_movies)-set(user_watch_list (User))

In [51]:
set(user_watch_list(User)).intersection(set(sim_movies))

{'Warrior (2011)'}

In [52]:
print (len(sim_movies)) 
print(len(user_watch_list (User))) 
print(len (not_watched))

124
29
123


In [53]:
#Getting the list of movies not watchedd by the user and to be recommended to the user
not_watched = list(not_watched)
not_watched

['Shame (2011)',
 'Great Gatsby, The (2013)',
 'Inside Llewyn Davis (2013)',
 'Certified Copy (Copie conforme) (2010)',
 'Phil Spector (2013)',
 'W. (2008)',
 'Behind the Candelabra (2013)',
 'Firm, The (2009)',
 'Short Term 12 (2013)',
 'Hitchcock (2012)',
 'Hunt, The (Jagten) (2012)',
 'Invictus (2009)',
 'Powder Blue (2009)',
 'Misfortunates, The (De helaasheid der dingen) (2009)',
 'Way Back, The (2010)',
 'Blind Side, The  (2009)',
 'Neds (2010)',
 'Moneyball (2011)',
 'Beautiful Boy (2010)',
 'Howl (2010)',
 'Class, The (Entre les murs) (2008)',
 'Vincent Wants to Sea (Vincent will meer) (2010)',
 'Frozen River (2008)',
 'Prayers for Bobby (2009)',
 'Blue Jasmine (2013)',
 'Precious (2009)',
 "I'm Here (2010)",
 'Fighter, The (2010)',
 'Rabbit Hole (2010)',
 'Tangerines (2013)',
 'Princess (Prinsessa) (2010)',
 'Social Network, The (2010)',
 'Detachment (2011)',
 'Albatross (2011)',
 "Barney's Version (2010)",
 "I Killed My Mother (J'ai tué ma mère) (2009)",
 'Legend No. 17 (2013

In [54]:
#Get the scores of the movies in the not_watched list from the similarity matrix WRT the selected movie

movie_name = np.array(movie_list.loc[movie_list.movieId==top[i], "title"])[0] 
cos_movies.loc[movie_name, not_watched]


title
Shame (2011)                              1.000000
Great Gatsby, The (2013)                  0.994988
Inside Llewyn Davis (2013)                0.994988
Certified Copy (Copie conforme) (2010)    0.999072
Phil Spector (2013)                       0.994988
                                            ...   
Frost/Nixon (2008)                        0.993097
Somers Town (2008)                        0.993097
Milk (2008)                               0.993097
Ides of March, The (2011)                 1.000000
White Frog (2012)                         0.998868
Name: Warrior (2011), Length: 123, dtype: float64

#### As we see the number of recommendations are too large for a user we create a function to limit the number of recommendations based on the choice of recommendations required.

In [55]:
#function to return the recommeded movies list
#arguments --> User : user id, N : top N preffered movies of the user
#              i : set any value of i (less than N) to get 1 of the movies from top N preffered movies of the user
#              sim : cosine similarity to check for the recommenadations. We need to vary the similarity to change the number of similar movies 
#              sim should be less than 1. 
#              choice : number of recommendation required by the user
def get_recommendation(User, N = 5, i = 1, sim = 0.99, choice = 10) :
    if (i < N) :         #to check the condition that i < N
    
        top = top_movies(User, N)         #getting the top N movies matched by a user

        sim_movies = get_similar_movies(top[i], sim)       #Get the most similar movie for the ith movie in the above List (LIST A) 

        watched = user_watch_list (User)               #Get the watchlist of the user (LIST B)

        #Get the list of movies in List A which are not in List B 
        #That is the movies in the similarity list which the user has not watched yet

        not_watched = set(sim_movies)-set(watched)        #not watched movies
        rec = list(not_watched)                           #converting to a list of not watched movies

        #we check if the number of unwatched movies is greater than the choice we recommend random choice of unique unwatched from unwatched movies list
        #else we recommend the unwatched list and some random movies from the unwatched list
        if len(rec) >= choice :
            return(list(np.random.choice(rec,choice,replace = False)))
        else :
            return(rec + list(np.random.choice(rec,choice - len(rec) ,replace = True)))
    else:
        return(print('Value of i should be less than N.'))

In [56]:
#getting recommendation for user with default arguments
get_recommendation(2)

['Louis Theroux: Law & Disorder (2008)',
 'Encounters at the End of the World (2008)',
 'Senna (2010)',
 'Burma VJ: Reporting from a Closed Country (Burma VJ: Reporter i et lukket land) (2008)',
 "Comic-Con Episode IV: A Fan's Hope (2011)",
 'With Great Power: The Stan Lee Story (2012)',
 'Cave of Forgotten Dreams (2010)',
 'It Might Get Loud (2008)',
 "Waiting for 'Superman' (2010)",
 'For the Bible Tells Me So (2007)']

In [57]:
#getting recommendation for user with N = 6, sim 0.99 and choice = 5 arguments
get_recommendation(5, 6, i = 1, sim = 0.99, choice = 5)

['Richard III (1995)',
 'Gallipoli (1981)',
 'Glory (1989)',
 'Tin Drum, The (Blechtrommel, Die) (1979)',
 'Breaker Morant (1980)']

In [58]:
#getting recommendation for user with N = 6, i = 4, sim 0.95 and choice = 20 arguments
get_recommendation(95, 6, i = 4, sim = 0.95, choice = 20)

['Strange Love of Martha Ivers, The (1946)',
 'Shadow of the Thin Man (1941)',
 'West Side Story (1961)',
 'Three Musketeers, The (1948)',
 'San Francisco (1936)',
 'Principal, The (1987)',
 'He Got Game (1998)',
 'Johnny Belinda (1948)',
 "Fellini's Casanova (Il Casanova di Federico Fellini) (1976)",
 'Walkabout (1971)',
 'Defiant Ones, The (1958)',
 'Requiem for a Dream (2000)',
 'Aguirre: The Wrath of God (Aguirre, der Zorn Gottes) (1972)',
 'To Live (Huozhe) (1994)',
 'Fireworks (Hana-bi) (1997)',
 'Mr. Skeffington (1944)',
 'Hot Spot, The (1990)',
 'Mogambo (1953)',
 'Sherlock Holmes: The Woman in Green (1945)',
 'Ju Dou (1990)']

### <a id = '6'>Part 2 : Movie Recommender System - (Collaborative Filtering)</a>
This is a user-user based collaborative filtering based movie recommendation model using the **K nearest neighbhors** algorithm.  

####   <b>[I have created a function get_knn_recommend() which will recommend movies to the user based on K nearest similar users.](#3)</b>

Similar users refers to the users who earlier watched the movies same as the current user and also given similar ratings to the movies.

##### In this part I am again reading the dataset from the csv files as I have made some alterations in the original dataset in the previous part.

### 1. Dataset :
#### 1.1 Movies

In [59]:
#Reading the movies data
movies = pd.read_csv(r"C:\Users\Lenovo\Documents\Codes\AML\Datasets\movies.csv")
movies.sample(2)

,movieId,title,genres
5027,7822,Mogambo (1953),Adventure|Drama|Romance
6794,60538,Shrooms (2007),Horror|Thriller


#### 1.2 Ratings

In [60]:
#Reading the ratings data
ratings = pd.read_csv(r"C:\Users\Lenovo\Documents\Codes\AML\Datasets\ratings.csv")
ratings.sample(2)

,userId,movieId,rating,timestamp
25190,177,2581,2.5,1435891169
57117,380,1882,3.0,1493667794


### 2. Data Preparation
#### 2.1 Creating a user - movie rating pivot table


In [61]:
#Creating a user - movie rating pivot table with user id as the rows and movie id as the columns.
#The ratings given by user to the movie is the value in the table.
table = ratings.pivot(index = 'userId', columns ='movieId', values = 'rating')
table.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 2.2 Making the table mean centred and replacing null values with zero.

In [62]:
#Subtracting the mean of the rows from the movie ratings and making the ratings mean centred.
table = (table.T.loc[:,:] - np.array( table.mean(axis=1))).T
table.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,-0.366379,NaN,-0.366379,NaN,NaN,-0.366379,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.363636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
#replacing null values with 0
table[table.isnull()] = 0
table.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,-0.366379,0.0,-0.366379,0.0,0.0,-0.366379,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.363636,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
table.describe()     

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
count,610.000000,610.000000,610.000000,610.000000,610.000000,610.000000,610.000000,610.000000,610.000000,610.000000,...,610.000000,610.000000,610.000000,610.000000,610.000000,610.000000,610.000000,610.000000,610.000000,610.000000
mean,0.110026,-0.009586,-0.020016,-0.012578,-0.041981,0.063284,-0.035474,-0.008197,-0.011946,-0.012189,...,-0.000336,-0.001156,0.000483,0.000483,-0.000336,0.000483,-0.000336,-0.000336,-0.000336,0.000610
std,0.488668,0.312009,0.261603,0.131636,0.259795,0.332932,0.286947,0.123147,0.145925,0.354453,...,0.008309,0.028554,0.011935,0.011935,0.008309,0.011935,0.008309,0.008309,0.008309,0.015063
min,-2.584034,-2.190283,-2.119681,-2.105263,-2.659292,-1.426087,-2.977427,-2.590909,-1.620690,-1.942122,...,-0.205224,-0.705224,0.000000,0.000000,-0.205224,0.000000,-0.205224,-0.205224,-0.205224,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.152629,1.520000,1.800000,0.000000,1.506369,2.402778,1.098214,0.447368,1.132231,1.739278,...,0.000000,0.000000,0.294776,0.294776,0.000000,0.294776,0.000000,0.000000,0.000000,0.372024


In [65]:
#creating a dataframe with movie Id and title from the movies data to get the movie name
movie_list = movies [["movieId", "title"]] 
movie_list.head() 

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


###  <a id = '3'>3. Recommendation using K - nearest neighbhors</a>

In [66]:
#function to calculate the distance betweeen two users 
def euclidean_dist (a,b) :                      #takes two matrices a and b as the input arguments
    dist = np.sqrt((b-a) @ np.transpose(b-a))   #calculating the distance between two inputs using vectorised operation 
    return(dist)

#### The below function will take the user id as the input and return the recommended movies for that user.  
#### The argument k specifies the number of nearest neighbhors to look for in the table. This value of k can varied according to the number of recommended movies required.
#### The choice argument is used to specify the number of movies to be recommended to the user.


In [67]:
#function to recommend movies to the user
#arguments --> user : user id, k : nunber of nearest neighbhors, choice : number of movies to recommended
def get_knn_recommend (user, k = 5, choice = 10) :

    distance = {}                                  #empty dictonary to store the distances between users
    row = table.loc[user,]                         #assigning current user to variable row
    for i in range (1,len(table)+1) :              #iterating the rows of the table
        dist = euclidean_dist(row,table.loc[i,])   #calculating the distance between ith and cureent user
        distance[i] = dist                         #storing the distance in the dictonary with user id

    #we sort the dict in ascending order on the distances and store the top k user ids in a temp variable    
    temp = sorted(distance.items(),key=lambda x:x[1])[1:k+1]   
    sim_user = list()                              #empty list to store the similar user ids
    for i in temp  :                               #iterating through the temp to extract the user ids
        sim_user.append(i[0])                      #appending the user ids in sim_user list

    watched = set()                                #empty set to store the movies watched by current useer
    for i in list(sim_user) :                      #iterating through the sim_user id list
        id=list(ratings.loc[ratings.userId==i, "movieId"])          #extracting the ids of movie watched by similar user
        empty = list()                             #empty list to store the watched movies by similar user
        for j in id :                              #iterating through the movie id list
            empty.append((movie_list.loc[movie_list.movieId == j, ["title"]].iloc[0,0]))  #appending the watched movie to empty list
        watched = set(watched).union(set(empty))   #taking the union of watched and emty list to get unique movies watched by similar users         

    id=list(ratings.loc[ratings.userId==user, "movieId"])        #extracting the ids of movie watched by current user
    empty = list()                                               #empty list to store the watched movies by current user
    for j in id :                                                #iterating through the movie id list
        empty.append((movie_list.loc[movie_list.movieId == j, ["title"]].iloc[0,0]))  #appending the watched movie to empty list
    watched_user = set(empty)                                    #creating a set of movies already watched by the user
    
    #the unwatched movies of the user are obtained by subtracting the watched movies of current user from the watched movies of similar users
    
    rec = list(set(watched)-set(watched_user))                   #storing the list unwatched movies by the user in a list
    
    #we check if the number of unwatched movies is greater than the choice we recommend random choice of unique unwatched from unwatched movies list
    #else we recommend the unwatched list and some random movies from the unwatched list
    if len(rec) >= choice :
        return(list(np.random.choice(rec,choice,replace = False)))
    else :
        return(rec + list(np.random.choice(rec,choice - len(rec) ,replace = True)))

In [68]:
#getting recommendation for user with default arguments
recommend = get_knn_recommend(5)
recommend

['Roman Holiday (1953)',
 'Despicable Me 2 (2013)',
 'Alien (1979)',
 'Die Hard: With a Vengeance (1995)',
 'Matrix, The (1999)',
 'District 9 (2009)',
 'Forrest Gump (1994)',
 'French Kiss (1995)',
 'While You Were Sleeping (1995)',
 'Red Violin, The (Violon rouge, Le) (1998)']

In [69]:
#getting recommendation for user from the 3 similar neighbhors
recommend = get_knn_recommend(53,3)
recommend

["Schindler's List (1993)",
 'Whiplash (2014)',
 'Silence of the Lambs, The (1991)',
 'Despicable Me (2010)',
 'Star Wars: The Last Jedi (2017)',
 'Interstellar (2014)',
 'Lord of the Rings: The Fellowship of the Ring, The (2001)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Groundhog Day (1993)',
 'Up (2009)']

In [70]:
#getting recommendation for user from the 1 similar neighbhor and recommendation list of 30 movies
recommend = get_knn_recommend(95,1,30)
recommend

['Immortal Beloved (1994)',
 'Red Violin, The (Violon rouge, Le) (1998)',
 'Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)',
 'Roman Holiday (1953)',
 'Arrival, The (1996)',
 'Pink Floyd: The Wall (1982)',
 'Dick Tracy (1990)',
 'When a Man Loves a Woman (1994)',
 'Kalifornia (1993)',
 'Island of Dr. Moreau, The (1996)',
 'Days of Thunder (1990)',
 'River Runs Through It, A (1992)',
 'Airheads (1994)',
 'Finding Forrester (2000)',
 'To Wong Foo, Thanks for Everything! Julie Newmar (1995)',
 'Return of the Pink Panther, The (1975)',
 'Ghost and the Darkness, The (1996)',
 'Halloween (1978)',
 'Duck Soup (1933)',
 'Benny & Joon (1993)',
 'Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)',
 'Red Violin, The (Violon rouge, Le) (1998)',
 'Kalifornia (1993)',
 'Duck Soup (1933)',
 'Island of Dr. Moreau, The (1996)',
 'To Wong Foo, Thanks for Everything! Julie Newmar (1995)',
 'Ghost and the Darkness, The (1996)',
 'When a Man Loves a Woman (1994)',
 'Dick Tracy (1990)',
 'Days of Thunder (1990)'